In [1]:
import numpy as np
import pandas as pd

In [2]:
offer = pd.read_csv('/kaggle/input/addtional-data/offer_metadata_converted_to_.csv')

In [3]:
test = pd.read_csv('/kaggle/input/to-import-features/test_cut_down.csv')
train = pd.read_csv('/kaggle/input/to-import-features/train_cut_down.csv')

In [4]:
offer.drop(columns=['Unnamed: 0','id9','f377','f374','id11'],inplace=True)

In [5]:
offer['f376'] = offer['f376'].fillna(0)
offer['id8'] = offer['id8'].fillna(10000000)

In [6]:
offer.isnull().any().any()

False

In [7]:
train = train.merge(offer, on='id3', how='left')
test = test.merge(offer,on='id3',how='left')

In [8]:
train['id10'].value_counts()

id10
1.0    688740
2.0     81324
Name: count, dtype: int64

In [9]:
train['f375'] = train['f375'].fillna(train['f375'].mode()[0])
train['f376'] = train['f376'].fillna(train['f376'].mode()[0])
train['id10'] = train['id10'].fillna(-1.0)
train['f378'] = train['f378'].fillna('__missing__')
train['id8'] = train['id8'].fillna(10000000)

test['f375'] = test['f375'].fillna(test['f375'].mode()[0])
test['f376'] = test['f376'].fillna(test['f376'].mode()[0])
test['id10'] = test['id10'].fillna(-1.0)
test['f378'] = test['f378'].fillna('__missing__')
test['id8'] = test['id8'].fillna(10000000)

In [10]:
train.drop(columns=['id12','id13'],inplace=True)
test.drop(columns=['id12','id13'],inplace=True)

In [11]:
train.isnull().any().any()

False

In [12]:
test.isnull().any().any()

False

In [13]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
cols=['f375', 'f376', 'id10', 'f378', 'id8']
mask=[False, False, True, True, True]
X = train[cols].copy()
y = train['y'].copy()
le = LabelEncoder()
X['id10'] = le.fit_transform(X['id10'].astype(str))
le = LabelEncoder()
X['f378'] = le.fit_transform(X['f378'].astype(str))
le = LabelEncoder()
X['id8'] = le.fit_transform(X['id8'].astype(str))
mi_scores = mutual_info_classif(X, y, discrete_features=mask, random_state=0)

In [14]:
mi_scores

array([0.11205761, 0.00625387, 0.00087431, 0.01815281, 0.00776332])

In [15]:
train.drop(columns=['id10'],inplace=True)
test.drop(columns=['id10'],inplace=True)

In [16]:
trans = pd.read_csv('/kaggle/input/addtional-data/add_trans_converted_to_.csv')

In [17]:
trans['id8'] = trans['id8'].fillna(10000000)

In [18]:
industry_avg = trans.groupby('id8')['f367'].mean().reset_index()
industry_avg.rename(columns={'f367': 'avg_trans_amt'}, inplace=True)
train = train.merge(industry_avg, on='id8', how='left')
test = test.merge(industry_avg, on='id8', how='left')

In [20]:
event = pd.read_csv('/kaggle/input/addtional-data/add_event_converted_to_.csv')

In [21]:
event['clicked'] = event['id7'].isnull().astype(int)

In [22]:
ctr_per_offer = event.groupby('id3')['clicked'].mean().reset_index()
ctr_per_offer.columns = ['id3','offer_ctr']
train = train.merge(ctr_per_offer, on='id3', how='left')
test = test.merge(ctr_per_offer, on='id3', how='left')

In [23]:
train.to_csv('train_to_feat_engg.csv',index=False)
test.to_csv('test_to_feat_engg.csv',index=False)

In [25]:
label = pd.read_csv('/kaggle/input/to-import-features/updated_label.csv')

In [26]:
for col in train.columns:
    if col not in label['masked_column'].to_list():
        print(col)

f375
f376
f378
id8
avg_trans_amt
offer_ctr


In [27]:
cols=['f375','f376','f378','id8','avg_trans_amt','offer_ctr']
desc=['redemption freq','discout rate','offerings body','CM industry code','avg trans amt','offer ctr']
types=['Numerical','Numerical','Categorical','Categorical','Numerical','Numerical']
to_add = pd.DataFrame({'masked_column':cols,'Description':desc,'Type':types})
label = pd.concat([label,to_add],axis=0)

In [28]:
label.to_csv('updated_labels2.csv',index=False)